In [0]:
-- Create SQL Procedure for ADS_RDS_CASE_OBJECT_STATUS_DISTRAINT_STATE ETL Process
CREATE OR REPLACE PROCEDURE gap_catalog.ads_etl_owner.sp_ADS_RDS_CASE_OBJECT_STATUS_DISTRAINT_STATE(
    p_load_date STRING,
    p_process_key STRING
)
LANGUAGE SQL
 SQL SECURITY INVOKER
COMMENT 'ETL procedure for ADS_RDS_CASE_OBJECT_STATUS_DISTRAINT_STATE - converted from notebook format'
AS
BEGIN
    -- Declare local variables
    DECLARE dif_table_name STRING DEFAULT 'gap_catalog.ads_etl_owner.DIFF_ADS_RDS_CASE_OBJECT_STATUS_DISTRAINT_STATE_ADS_MAP_SCD_DIFF';
    DECLARE my_log_id STRING;
    BEGIN
        CALL gap_catalog.log.start_performance_test(
            test_case_name => 'ADS_RDS_PROC',
            step_name => 'ADS_RDS_CASE_OBJECT_STATUS_DISTRAINT_STATE',
            psource_sys_origin  => 'rds_distraintstate',
            process_key => p_process_key,
            target_table => 'CASE_OBJECT_STATUS',
            log_id => my_log_id,
            p_load_date => p_load_date
        );

        /* Declare variables */ 
        --DECLARE OR REPLACE VARIABLE map_id STRING DEFAULT 'ADS_RDS_CASE_OBJECT_STATUS_DISTRAINT_STATE (Fixed)';
        --DECLARE OR REPLACE VARIABLE max_key BIGINT DEFAULT 0;

        -- Get the maximum COS_KEY from Target table
        /*SET VAR max_key = (
            SELECT COALESCE(MAX(cos_key), 0) 
            FROM gap_catalog.ads_owner.case_object_status
        );
        */
        TRUNCATE TABLE gap_catalog.ads_etl_owner.XC_RDS_CASE_OBJECT_STATUS_DISTRAINT_STATE;
        
        /* Fill XC Table */
        INSERT INTO gap_catalog.ads_etl_owner.XC_RDS_CASE_OBJECT_STATUS_DISTRAINT_STATE 
              with C_0FILTERADCSLAAK138GRG1J678E1NG6FA as (
              select
                  RDS_DISTRAINTSTATE.id  ID,
                  RDS_DISTRAINTSTATE.value  VALUE
                from
                  gap_catalog.ads_etl_owner.DLK_ADS_LOV_RDS_DISTRAINTSTATE RDS_DISTRAINTSTATE
                where ( CAST(from_utc_timestamp(SYS_EFFECTIVE_DATE, 'Europe/Prague') AS DATE) = CAST(p_load_date AS DATE)
                    and RDS_DISTRAINTSTATE.std_deleted_flag = 'N'
                    and RDS_DISTRAINTSTATE.sys = 'Brasil'
                    and RDS_DISTRAINTSTATE.lang = 'CZ'
                      )
              )
              select  /*+no hint*/
                      /*seznam vkladanych nebo updatovanych sloupcu bez SK, auditnich atribudu a deleted flagu*/
                  FILTER_A.ID as COS_SOURCE_ID,
                  'RDS' as COS_SOURCE_SYSTEM_ID,
                  'rds_distraintstate' as COS_SOURCE_SYS_ORIGIN,
                  FILTER_A.VALUE as COS_DESC,
                  'N' as COS_DELETED_FLAG
                from (C_0FILTERADCSLAAK138GRG1J678E1NG6FA FILTER_A)
                where (1=1)
        ;

        /* Cleanup DIFF Table */
        DROP TABLE IF EXISTS gap_catalog.ads_etl_owner.DIFF_ADS_RDS_CASE_OBJECT_STATUS_DISTRAINT_STATE_ADS_MAP_SCD_DIFF;

        /* Create DIFF Table */
        create  table gap_catalog.ads_etl_owner.DIFF_ADS_RDS_CASE_OBJECT_STATUS_DISTRAINT_STATE_ADS_MAP_SCD_DIFF
        ( tech_del_flg  char(1),
          tech_new_rec  char(1),
          tech_rid      varchar(255),
          COS_KEY  decimal(38,0),
          COS_KEY_NEW BIGINT GENERATED ALWAYS AS IDENTITY (START WITH 1 INCREMENT BY 1),
          COS_SOURCE_ID  varchar(120),
          COS_SOURCE_SYSTEM_ID  varchar(120),
          COS_SOURCE_SYS_ORIGIN  varchar(120),
          COS_DESC  varchar(400))
        ;

        /* Populate DIFF Table - New/Updated Records */
        insert into gap_catalog.ads_etl_owner.DIFF_ADS_RDS_CASE_OBJECT_STATUS_DISTRAINT_STATE_ADS_MAP_SCD_DIFF
        (
          tech_del_flg,
          tech_new_rec,
          tech_rid,
          COS_KEY,
          COS_SOURCE_ID,
          COS_SOURCE_SYSTEM_ID,
          COS_SOURCE_SYS_ORIGIN,
          COS_DESC
        )
        select   'N' as tech_del_flg, 
            case when trg.COS_SOURCE_ID is null then 'Y' else 'N' end as tech_new_rec,
            trg.rid as tech_rid,
           trg.COS_KEY, 
           src.COS_SOURCE_ID, 
           src.COS_SOURCE_SYSTEM_ID, 
           src.COS_SOURCE_SYS_ORIGIN, 
           src.COS_DESC
         from 
            (select /*+ full(xc) */      COS_SOURCE_ID, 
               COS_SOURCE_SYSTEM_ID, 
               COS_SOURCE_SYS_ORIGIN, 
               COS_DESC, 
               COS_DELETED_FLAG
               from gap_catalog.ads_etl_owner.XC_RDS_CASE_OBJECT_STATUS_DISTRAINT_STATE xc
            where (COS_SOURCE_SYS_ORIGIN = 'rds_distraintstate'
                  and COS_SOURCE_SYSTEM_ID = 'RDS')) src LEFT JOIN
            (select  cos_key||'.'||cos_valid_from as rid, t.* from gap_catalog.ads_owner.CASE_OBJECT_STATUS t
              where COS_CURRENT_FLAG  = 'Y'
                and COS_VALID_TO  = DATE('3000-01-01')
               and (COS_SOURCE_SYS_ORIGIN = 'rds_distraintstate'
                  and COS_SOURCE_SYSTEM_ID = 'RDS')      ) trg
        ON trg.COS_SOURCE_ID = src.COS_SOURCE_ID
         and trg.COS_SOURCE_SYSTEM_ID = src.COS_SOURCE_SYSTEM_ID
         and trg.COS_SOURCE_SYS_ORIGIN = src.COS_SOURCE_SYS_ORIGIN
         and trg.COS_VALID_TO = DATE('3000-01-01') WHERE (
             decode( src.COS_DESC,trg.COS_DESC,1,0 ) = 0  or
             decode( src.COS_DELETED_FLAG,trg.COS_DELETED_FLAG,1,0 ) = 0 or 
             trg.COS_SOURCE_ID is null   );

        /* Populate DIFF Table - Deleted Records */
        insert into gap_catalog.ads_etl_owner.DIFF_ADS_RDS_CASE_OBJECT_STATUS_DISTRAINT_STATE_ADS_MAP_SCD_DIFF
        (
          tech_del_flg,
          tech_new_rec,
          tech_rid,
          COS_KEY,
          COS_SOURCE_ID,
          COS_SOURCE_SYSTEM_ID,
          COS_SOURCE_SYS_ORIGIN,
          COS_DESC
        )
        select   'Y' as tech_del_flg, 
         'N' as tech_new_rec, 
         trg.rid as tech_rid, 
           trg.COS_KEY, 
           trg.COS_SOURCE_ID, 
           trg.COS_SOURCE_SYSTEM_ID, 
           trg.COS_SOURCE_SYS_ORIGIN, 
           trg.COS_DESC
         from 
            (select  cos_key||'.'||cos_valid_from as rid, t.* from gap_catalog.ads_owner.CASE_OBJECT_STATUS t
              where COS_CURRENT_FLAG  = 'Y' and COS_DELETED_FLAG  = 'N'
                and COS_VALID_TO  = DATE('3000-01-01')
               and (COS_SOURCE_SYS_ORIGIN = 'rds_distraintstate'
                  and COS_SOURCE_SYSTEM_ID = 'RDS')      ) trg LEFT JOIN
            (select /*+ full(xc) */      COS_SOURCE_ID, 
               COS_SOURCE_SYSTEM_ID, 
               COS_SOURCE_SYS_ORIGIN, 
               DATE('3000-01-01') as COS_VALID_TO
               from gap_catalog.ads_etl_owner.XC_RDS_CASE_OBJECT_STATUS_DISTRAINT_STATE xc
            where (COS_SOURCE_SYS_ORIGIN = 'rds_distraintstate'
                  and COS_SOURCE_SYSTEM_ID = 'RDS')) src
        ON trg.COS_SOURCE_ID = src.COS_SOURCE_ID
         and trg.COS_SOURCE_SYSTEM_ID = src.COS_SOURCE_SYSTEM_ID
         and trg.COS_SOURCE_SYS_ORIGIN = src.COS_SOURCE_SYS_ORIGIN
         and trg.COS_VALID_TO = src.COS_VALID_TO WHERE (src.COS_SOURCE_ID is null);

        /* Close Old Records in Target */
        update  gap_catalog.ads_owner.CASE_OBJECT_STATUS set
               COS_UPDATED_DATETIME = CURRENT_TIMESTAMP(),
               COS_UPDATE_PROCESS_KEY = 13173833,
               COS_CURRENT_FLAG = 'N', 
               COS_VALID_TO = DATEADD(DAY, -1, CAST(p_load_date AS DATE))
         where COS_CURRENT_FLAG = 'Y'
           and COS_VALID_TO = DATE('3000-01-01')
           and cos_key||'.'||cos_valid_from in (select tech_rid from gap_catalog.ads_etl_owner.DIFF_ADS_RDS_CASE_OBJECT_STATUS_DISTRAINT_STATE_ADS_MAP_SCD_DIFF where tech_rid is not null);

        /* Insert Changed Records */
        insert  into gap_catalog.ads_owner.CASE_OBJECT_STATUS 
         ( COS_KEY, 
           COS_SOURCE_ID, 
           COS_SOURCE_SYSTEM_ID, 
           COS_SOURCE_SYS_ORIGIN, 
           COS_DESC, 
           COS_VALID_FROM, 
           COS_VALID_TO, 
           COS_CURRENT_FLAG, 
           COS_DELETED_FLAG, 
           COS_INSERTED_DATETIME, 
           COS_INSERT_PROCESS_KEY, 
           COS_UPDATED_DATETIME, 
           COS_UPDATE_PROCESS_KEY)
        select COS_KEY, 
           COS_SOURCE_ID, 
           COS_SOURCE_SYSTEM_ID, 
           COS_SOURCE_SYS_ORIGIN, 
           COS_DESC, 
           CAST(p_load_date AS DATE) as COS_VALID_FROM, 
           DATE('3000-01-01') as COS_VALID_TO, 
           'Y' as COS_CURRENT_FLAG, 
           tech_del_flg as COS_DELETED_FLAG, 
           CURRENT_TIMESTAMP() as COS_INSERTED_DATETIME, 
           CAST(p_process_key AS BIGINT) as COS_INSERT_PROCESS_KEY, 
           CURRENT_TIMESTAMP() as COS_UPDATED_DATETIME, 
           CAST(p_process_key AS BIGINT) as COS_UPDATE_PROCESS_KEY
          from gap_catalog.ads_etl_owner.DIFF_ADS_RDS_CASE_OBJECT_STATUS_DISTRAINT_STATE_ADS_MAP_SCD_DIFF
         where tech_new_rec = 'N';

        /* Insert New Records */
        insert  into gap_catalog.ads_owner.CASE_OBJECT_STATUS 
         ( COS_KEY, 
           COS_SOURCE_ID, 
           COS_SOURCE_SYSTEM_ID, 
           COS_SOURCE_SYS_ORIGIN, 
           COS_DESC, 
           COS_VALID_FROM, 
           COS_VALID_TO, 
           COS_CURRENT_FLAG, 
           COS_DELETED_FLAG, 
           COS_INSERTED_DATETIME, 
           COS_INSERT_PROCESS_KEY, 
           COS_UPDATED_DATETIME, 
           COS_UPDATE_PROCESS_KEY)
        select 
           --COS_KEY_NEW + max_key AS COS_KEY,  -- Add the offset to maintain key sequence, 
           xxhash64(uuid()) AS COS_KEY,
           COS_SOURCE_ID, 
           COS_SOURCE_SYSTEM_ID, 
           COS_SOURCE_SYS_ORIGIN, 
           COS_DESC, 
           CAST(p_load_date AS DATE) as COS_VALID_FROM, 
           DATE('3000-01-01') as COS_VALID_TO, 
           'Y' as COS_CURRENT_FLAG, 
           tech_del_flg as COS_DELETED_FLAG, 
           CURRENT_TIMESTAMP() as COS_INSERTED_DATETIME, 
           CAST(p_process_key AS BIGINT) as COS_INSERT_PROCESS_KEY, 
           CURRENT_TIMESTAMP() as COS_UPDATED_DATETIME, 
           CAST(p_process_key AS BIGINT) as COS_UPDATE_PROCESS_KEY
          from gap_catalog.ads_etl_owner.DIFF_ADS_RDS_CASE_OBJECT_STATUS_DISTRAINT_STATE_ADS_MAP_SCD_DIFF
         where tech_new_rec = 'Y';

        CALL gap_catalog.log.complete_performance_test(
            plog_id => my_log_id
        );

    END;
    
END;